# Missing Persons FOIA Cleaning challenge

In [331]:
# Introductory imports
import pandas as pd
from datetime import datetime
import numpy as np
import re
raw_df = pd.read_csv("chi_mp.csv", delimiter="|")
cleaned_df = pd.DataFrame()
raw_df.head()

,RD,date,age,age_unit,race,gender,stnumber,stdir,st
0,F289698,15-May-00,15.0,NaN,BLACK,F,10XX,W,87 ST
1,F328277,31-May-00,28.0,NaN,BLACK,M,79XX,S,VINCENNES AV
2,F382937,25-Jun-00,46.0,NaN,HISPANIC,M,77XX,S,SANGAMON ST
3,F383892,26-Jun-00,18.0,NaN,BLACK,M,61XX,S,MAPLEWOOD AV
4,F582625,19-Sep-00,7.0,Y,HISPANIC,M,38XX,N,CHRISTIANA AV


## Date

We can see in the original PDF this data was pulled by CPD on August 4, 2016, and includes open missing persons cases as of August 1, 2016, dating back to 2000.

We reformat and move the date into the final dataframe, and can see the data spans from May 15, 2000 to August 1, 2016. This matches our understanding of what data we requested.

In [332]:
cleaned_df["last_seen"] = pd.to_datetime(raw_df["date"]).dt.strftime('%Y-%m-%d')

In [333]:
min(cleaned_df["last_seen"]), max(cleaned_df["last_seen"])

('2000-05-15', '2016-08-01')

## Race

> Would you combine any of the racial categories provided in preparation for analysis? What would you do with the NA’s? How did you come to this decision?  


To start cleaning the race data, it's helpful to see what race categories are represented and in what amounts. It's additionally good to see how many NaN/null values there are for race.

In the first cell, we see that the groupings are "Black", "White", "Hispanic", "Unknown", "Asian/Pacific Islander", "American Indian/ Alaskan Native". Something to notice here is that "Hispanic" is included as a racial category instead of an ethnicity that could also be combined with race, as in the census (i.e., on the census you can put your race as Black and your ethnicity as Hispanic/Latinx). 

This difference in categorization seems important to note, especially if we wanted to compare these groups to census data,  for example to show that Black people are overrepresented in this data and white people are underrepresented. But these doesn't seem to me to be a way or need to combine any of these categories.

In the second cell, we see that there are 37 entries for whom race is null. Something to note, which we can quickly observe in the third cell, is that often times race data is missing in conjunction which other demographic information like age and gender. Additionally, we can see that the vast majority of people with missing race data were "last seen" in 2016 in the months preceeding August 1, when the data was pulled.

I take this to mean that this data is not necessarily collected when a missing person is reported, but is collected through the process of trying to find said person.

I would recommend we simply recode all NaN/null values as "UNKNOWN" so we do not exclude these 37 people from the dataset, but note the lack of data in their cases.

In the final cell we clean the data, replacing the 37 null values as "UNKNOWN", bringing the total of people with "UNKNOWN" race to 39.




In [338]:
raw_df.groupby("race").count()

,RD,date,age,age_unit,gender,stnumber,stdir,st
race,,,,,,,,
AMER IND/ALASKAN NATIVE,1,1,1,1,1,1,1,1
ASIAN/PACIFIC ISLANDER,11,11,11,11,11,11,11,11
BLACK,492,492,492,489,487,492,492,492
HISPANIC,135,135,134,133,134,135,135,135
UNKNOWN,2,2,1,2,2,2,2,2
WHITE,151,151,149,149,150,151,151,151


In [339]:
raw_df[raw_df["race"].isnull()].count()

RD          37
date        37
age          3
age_unit     4
race         0
gender       1
stnumber    37
stdir       37
st          37
dtype: int64

In [340]:
raw_df[raw_df["race"].isnull()].head()

,RD,date,age,age_unit,race,gender,stnumber,stdir,st
44,HN389421,6-Jun-07,81.0,Y,NaN,NaN,124XX,S,NORMAL AVE
79,HP370305,1-Jun-08,NaN,NaN,NaN,NaN,16XX,W,GRAND AVE
89,HR131055,20-Jan-09,NaN,NaN,NaN,NaN,15XX,W,19TH ST
97,HS193468,5-Mar-10,NaN,NaN,NaN,NaN,61XX,N,TALMAN AVE
163,HZ274100,21-May-16,52.0,Y,NaN,M,53XX,N,WESTERN AVE


In [341]:
# This row cleans the data
cleaned_df["race"] = raw_df["race"]
cleaned_df["race"].fillna("UNKNOWN", inplace=True)
cleaned_df.groupby("race").count()

,last_seen
race,
AMER IND/ALASKAN NATIVE,1
ASIAN/PACIFIC ISLANDER,11
BLACK,492
HISPANIC,135
UNKNOWN,39
WHITE,151


# Gender / Sex

To clean this data, we retitle "gender" to "sex," coerce NaN/null values to "unknown," and do some cleaning where we remove digits from the address data that have leaked into the "gender" column, most likely through the OCR process.

> How many many of the missing persons cases involve a Male? Female? Unknown?

The second cell answers this question: there are 390 female missing persons, 395 male missing persons, and 1 unknown.

> What can we learn from this column?  Are there any limitations based on the information provided? 

For limitations based on the data, we should clearly note that many people would self-report their gender as outside "male" or "female". Additionally, this data does not capture if people are cis or trans; in light of these exclusions there is a strong possibility that some people appearing in this data would be misgendered by police, or even by family or associates who provided this data.

In terms of what we can learn from the column, the near equivalence of male and female representation might be a surprise to some. Grouping by both race and sex is useful (third cell). We see that the gender breakdown is not perfectly consistent across race: there are more missing Black women than Black men but more missing white men than white women. More statistical analysis could tell us if this breakdown is statistically significant.


In [349]:
# This cell cleans the data
cleaned_df["sex"] = raw_df["gender"]
# This row cleans the erroneous address data that has leaked into the "sex" column
cleaned_df["sex"] = cleaned_df["sex"].str.split(" ").str[0]
unknown_sex_index = cleaned_df.loc[(cleaned_df["sex"] != "M") & (cleaned_df["sex"] != "F")].index
cleaned_df.loc[unknown_sex_index]["sex"] = "unknown"

In [348]:
cleaned_df.groupby("sex").count()

,last_seen,race
sex,,
F,390,390
M,395,395
X,1,1


In [350]:
cleaned_df.groupby(["sex", "race"]).count()

last_seen
sex race                              
F   AMER IND/ALASKAN NATIVE          1
    ASIAN/PACIFIC ISLANDER           5
    BLACK                          253
    HISPANIC                        69
    WHITE                           62
M   ASIAN/PACIFIC ISLANDER           6
    BLACK                          234
    HISPANIC                        65
    UNKNOWN                          2
    WHITE                           88
X   UNKNOWN                          1

# Address

We use an intermediate dataframe to clean each of the address fields before including in the final dataframe (second cell). As mentioned in the "gender" section above, as an artifact of OCR some address information appeared in the "gender" column: here we extract that address data via regex and add it to the street address provided. With more time, this cleaning could be refined a bit: you can see there is an "M" (as in "male") that appears in the final row's address.

Address data offers an easy opportunity for geographic analysis, but that's a bit beyond the scope of this data cleaning.

In [304]:
intermediate_df = pd.DataFrame()
intermediate_df["misplaced_address"] = raw_df["gender"].str.extract('(\d+)')[0]
intermediate_df["misplaced_address"].fillna("", inplace=True)
intermediate_df["stnumber"] = raw_df["stnumber"].str.replace("X", "0")
intermediate_df["stdir"] = raw_df["stdir"]
intermediate_df["st"] = raw_df["st"]
cleaned_df["address"] = intermediate_df["misplaced_address"] + intermediate_df["stnumber"] + " " + intermediate_df["stdir"] + " " + intermediate_df["st"] + " Chicago, IL"

In [305]:
cleaned_df

,last_seen,race,sex,age,address
0,2000-05-15,BLACK,F,15.0,"1000 W 87 ST Chicago, IL"
1,2000-05-31,BLACK,M,28.0,"7900 S VINCENNES AV Chicago, IL"
2,2000-06-25,HISPANIC,M,46.0,"7700 S SANGAMON ST Chicago, IL"
3,2000-06-26,BLACK,M,18.0,"6100 S MAPLEWOOD AV Chicago, IL"
4,2000-09-19,HISPANIC,M,7.0,"3800 N CHRISTIANA AV Chicago, IL"
...,...,...,...,...,...
824,2016-08-01,NaN,NaN,NaN,"3600 W FILLMORE ST Chicago, IL"
825,2016-08-01,NaN,NaN,NaN,"4500 N BEACON ST Chicago, IL"
826,2016-08-01,NaN,NaN,NaN,"6100 W BARRY AVE Chicago, IL"
827,2016-08-01,NaN,NaN,NaN,"800 W WELLINGTON AVE Chicago, IL"


# Final analysis

> What would you do if you had more time? 
What information could you learn from this dataset? What questions do you have about the data? If you had more time, what would you do with this dataset?
If you weren't able to finish the cleaning--can you describe what you would have done to accomplish the coding exercise? 


I think a limitation of this dataset is that, since it reports **open** missing persons cases, there is some crucial information about policing that it does not examine. It would be hugely useful to compare this database to the database of **closed** missing persons cases, in order to find if certain genders, races, or ages differ between who goes missing and who is eventually found. Along those lines, we might segment this dataset into looking at cases that are more than, say, 1 year old, as a proxy for "unsolved."


If I had more time, I would want to pay more attention to the age category, to see if the age distribution is consistent across race or gender. 